# EPL Matches Score Prediction

## Imports and Reading

In [264]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn import neural_network

In [265]:
df_2009_10 = pd.read_csv('2009-10.csv')
df_2010_11=pd.read_csv("2010-11.csv")
df_2011_12=pd.read_csv("2011-12.csv")
df_2012_13=pd.read_csv("2012-13.csv")
df_2013_14=pd.read_csv("2013-14.csv")
df_2014_15=pd.read_csv("2014-15.csv")
df_2015_16=pd.read_csv("2015-16.csv")
df_2016_17=pd.read_csv("2016-17.csv")
df_2017_18=pd.read_csv("2017-18.csv")
df_2018_19=pd.read_csv("2018-19.csv")
df_2019_20=pd.read_csv("2019-20.csv")

## B365

In [266]:
def bet365(df):
    B365=df[['B365H','B365A',"B365D"]]
    return B365

In [267]:
bet365_2009_10=bet365(df_2009_10)
bet365_2010_11=bet365(df_2010_11)
bet365_2011_12=bet365(df_2011_12)
bet365_2012_13=bet365(df_2012_13)
bet365_2013_14=bet365(df_2013_14)
bet365_2014_15=bet365(df_2014_15)
bet365_2015_16=bet365(df_2015_16)
bet365_2016_17=bet365(df_2016_17)
bet365_2017_18=bet365(df_2017_18)
bet365_2018_19=bet365(df_2018_19)
bet365_2019_20=bet365(df_2019_20)

## Goals scored

In [269]:
def home_away_goals(df):    
    home_points=np.zeros(df.shape[0])
    away_points=np.zeros(df.shape[0])
    for i in df.index:
        home_team=df.loc[i,"HomeTeam"]
        away_team=df.loc[i,"AwayTeam"]
        temp1=df.groupby(["AwayTeam"]).get_group((f"{home_team}"))["FTAG"]
        temp2=df.groupby(["HomeTeam"]).get_group((f"{home_team}"))["FTHG"]
        temp3=pd.concat([temp1,temp2]).sort_index()
        temp4=temp3.loc[temp3.index[0]:i]
        temp5=temp4.loc[temp4.index[0]:temp4.index[len(temp4)-2]]
        if len(temp5)==1:
            home_points[i]=0
        else:
            home_points[i]=temp5.sum()
        temp5=df.groupby(["AwayTeam"]).get_group((f"{away_team}"))["FTAG"]
        temp6=df.groupby(["HomeTeam"]).get_group((f"{away_team}"))["FTHG"]
        temp7=pd.concat([temp5,temp6]).sort_index()
        temp8=temp7.loc[temp7.index[0]:i]
        temp9=temp8.loc[temp8.index[0]:temp8.index[len(temp8)-2]]
        if len(temp9)==1:
            away_points[i]=0
        else:
            away_points[i]=temp9.sum()
    return pd.DataFrame({"home goal scored":home_points,"away goal scored":away_points})

In [270]:
home_away_goals_2009_10=home_away_goals(df_2009_10)
home_away_goals_2010_11=home_away_goals(df_2010_11)
home_away_goals_2011_12=home_away_goals(df_2011_12)
home_away_goals_2012_13=home_away_goals(df_2012_13)
home_away_goals_2013_14=home_away_goals(df_2013_14)
home_away_goals_2014_15=home_away_goals(df_2014_15)
home_away_goals_2015_16=home_away_goals(df_2015_16)
home_away_goals_2016_17=home_away_goals(df_2016_17)
home_away_goals_2017_18=home_away_goals(df_2017_18)
home_away_goals_2018_19=home_away_goals(df_2018_19)
home_away_goals_2019_20=home_away_goals(df_2019_20)

## Form

In [272]:
def form(df):    
    home_form=np.zeros(df.shape[0])
    away_form=np.zeros(df.shape[0])
    for i in df.index:
        home_team=df.loc[i,"HomeTeam"]
        away_team=df.loc[i,"AwayTeam"]
        temp1=df.groupby(["AwayTeam"]).get_group((f"{home_team}"))["FTR"].replace({"H":0,"A":3,"D":1})
        temp2=df.groupby(["HomeTeam"]).get_group((f"{home_team}"))["FTR"].replace({"H":3,"A":0,"D":1})
        temp3=pd.concat([temp1,temp2]).sort_index()
        temp4=temp3.loc[temp3.index[0]:i]
        if len(temp4)==1:
            home_form[i]=0
        elif len(temp4)<=5:
            home_form[i]=temp4.loc[temp4.index[0]:temp4.index[len(temp4)-2]].sum()
        else:
            last_5_home=temp4.tail(6)
            home_form[i]=last_5_home.loc[last_5_home.index[0]:last_5_home.index[4]].sum()
        temp5=df.groupby(["AwayTeam"]).get_group((f"{away_team}"))["FTR"].replace({"H":0,"A":3,"D":1})
        temp6=df.groupby(["HomeTeam"]).get_group((f"{away_team}"))["FTR"].replace({"H":3,"A":0,"D":1})
        temp7=pd.concat([temp5,temp6]).sort_index()
        temp8=temp7.loc[temp7.index[0]:i]
        if len(temp8)==1:
            away_form[i]=0
        elif len(temp8)<=5:
            away_form[i]=temp8.loc[temp8.index[0]:temp8.index[len(temp8)-2]].sum()
        else:
            last_5_away=temp8.tail(6)
            away_form[i]=last_5_away.loc[last_5_away.index[0]:last_5_away.index[4]].sum()
    return pd.DataFrame({"home form":home_form,"away form":away_form})

In [273]:
form_2009_10=form(df_2009_10)
form_2010_11=form(df_2010_11)
form_2011_12=form(df_2011_12)
form_2012_13=form(df_2012_13)
form_2013_14=form(df_2013_14)
form_2014_15=form(df_2014_15)
form_2015_16=form(df_2015_16)
form_2016_17=form(df_2016_17)
form_2017_18=form(df_2017_18)
form_2018_19=form(df_2018_19)
form_2019_20=form(df_2019_20)

## Points

In [275]:
def points(df):    
    home_points=np.zeros(df.shape[0])
    away_points=np.zeros(df.shape[0])
    for i in df.index:
        home_team=df.loc[i,"HomeTeam"]
        away_team=df.loc[i,"AwayTeam"]
        temp1=df.groupby(["AwayTeam"]).get_group((f"{home_team}"))["FTR"].replace({"H":0,"A":3,"D":1})
        temp2=df.groupby(["HomeTeam"]).get_group((f"{home_team}"))["FTR"].replace({"H":3,"A":0,"D":1})
        temp3=pd.concat([temp1,temp2]).sort_index()
        temp4=temp3.loc[temp3.index[0]:i]
        temp5=temp4.loc[temp4.index[0]:temp4.index[len(temp4)-2]]
        if len(temp5)==1:
            home_points[i]=0
        else:
            home_points[i]=temp5.sum()
        temp5=df.groupby(["AwayTeam"]).get_group((f"{away_team}"))["FTR"].replace({"H":0,"A":3,"D":1})
        temp6=df.groupby(["HomeTeam"]).get_group((f"{away_team}"))["FTR"].replace({"H":3,"A":0,"D":1})
        temp7=pd.concat([temp5,temp6]).sort_index()
        temp8=temp7.loc[temp7.index[0]:i]
        temp9=temp8.loc[temp8.index[0]:temp8.index[len(temp8)-2]]
        if len(temp9)==1:
            away_points[i]=0
        else:
            away_points[i]=temp9.sum()
    return pd.DataFrame({"home points":home_points,"away points":away_points})

In [276]:
points_2009_10=points(df_2009_10)
points_2010_11=points(df_2010_11)
points_2011_12=points(df_2011_12)
points_2012_13=points(df_2012_13)
points_2013_14=points(df_2013_14)
points_2014_15=points(df_2014_15)
points_2015_16=points(df_2015_16)
points_2016_17=points(df_2016_17)
points_2017_18=points(df_2017_18)
points_2018_19=points(df_2018_19)
points_2019_20=points(df_2019_20)

## Goals Conceded

In [278]:
def goals_conceded(df):    
    home_points=np.zeros(df.shape[0])
    away_points=np.zeros(df.shape[0])
    for i in df.index:
        home_team=df.loc[i,"HomeTeam"]
        away_team=df.loc[i,"AwayTeam"]
        temp1=df.groupby(["AwayTeam"]).get_group((f"{home_team}"))["FTHG"]
        temp2=df.groupby(["HomeTeam"]).get_group((f"{home_team}"))["FTAG"]
        temp3=pd.concat([temp1,temp2]).sort_index()
        temp4=temp3.loc[temp3.index[0]:i]
        temp5=temp4.loc[temp4.index[0]:temp4.index[len(temp4)-2]]
        if len(temp5)==1:
            home_points[i]=0
        else:
            home_points[i]=temp5.sum()
        temp5=df.groupby(["AwayTeam"]).get_group((f"{away_team}"))["FTHG"]
        temp6=df.groupby(["HomeTeam"]).get_group((f"{away_team}"))["FTAG"]
        temp7=pd.concat([temp5,temp6]).sort_index()
        temp8=temp7.loc[temp7.index[0]:i]
        temp9=temp8.loc[temp8.index[0]:temp8.index[len(temp8)-2]]
        if len(temp9)==1:
            away_points[i]=0
        else:
            away_points[i]=temp9.sum()
    return pd.DataFrame({"home goal conceded":home_points,"away goal conceded":away_points})

In [279]:
goals_conceded_2009_10=goals_conceded(df_2009_10)
goals_conceded_2010_11=goals_conceded(df_2010_11)
goals_conceded_2011_12=goals_conceded(df_2011_12)
goals_conceded_2012_13=goals_conceded(df_2012_13)
goals_conceded_2013_14=goals_conceded(df_2013_14)
goals_conceded_2014_15=goals_conceded(df_2014_15)
goals_conceded_2015_16=goals_conceded(df_2015_16)
goals_conceded_2016_17=goals_conceded(df_2016_17)
goals_conceded_2017_18=goals_conceded(df_2017_18)
goals_conceded_2018_19=goals_conceded(df_2018_19)
goals_conceded_2019_20=goals_conceded(df_2019_20)

## Red Card

In [281]:
def red_card_last_game(df):    
    home_points=np.zeros(df.shape[0])
    away_points=np.zeros(df.shape[0])
    for i in df.index:
        home_team=df.loc[i,"HomeTeam"]
        away_team=df.loc[i,"AwayTeam"]
        temp1=df.groupby(["AwayTeam"]).get_group((f"{home_team}"))["AR"]
        temp2=df.groupby(["HomeTeam"]).get_group((f"{home_team}"))["HR"]
        temp3=pd.concat([temp1,temp2]).sort_index()
        temp4=temp3.loc[temp3.index[0]:i]
        temp5=temp4.loc[temp4.index[0]:temp4.index[len(temp4)-2]]
        if len(temp5)==1:
            home_points[i]=0
        else:
            home_points[i]=temp5.loc[temp5.index[len(temp5)-1]]
        temp5=df.groupby(["AwayTeam"]).get_group((f"{away_team}"))["AR"]
        temp6=df.groupby(["HomeTeam"]).get_group((f"{away_team}"))["HR"]
        temp7=pd.concat([temp5,temp6]).sort_index()
        temp8=temp7.loc[temp7.index[0]:i]
        temp9=temp8.loc[temp8.index[0]:temp8.index[len(temp8)-2]]
        if len(temp9)==1:
            away_points[i]=0
        else:
            away_points[i]=temp9.loc[temp9.index[len(temp9)-1]]
    return pd.DataFrame({"home red card in last match":home_points,"away red card in last match":away_points})

In [282]:
red_card_last_game_2009_10=red_card_last_game(df_2009_10)
red_card_last_game_2010_11=red_card_last_game(df_2010_11)
red_card_last_game_2011_12=red_card_last_game(df_2011_12)
red_card_last_game_2012_13=red_card_last_game(df_2012_13)
red_card_last_game_2013_14=red_card_last_game(df_2013_14)
red_card_last_game_2014_15=red_card_last_game(df_2014_15)
red_card_last_game_2015_16=red_card_last_game(df_2015_16)
red_card_last_game_2016_17=red_card_last_game(df_2016_17)
red_card_last_game_2017_18=red_card_last_game(df_2017_18)
red_card_last_game_2018_19=red_card_last_game(df_2018_19)
red_card_last_game_2019_20=red_card_last_game(df_2019_20)

## Bet and Win

In [284]:
def bw(df):
    BW=df[['BWH','BWA',"BWD"]]
    return BW

In [285]:
bw_2009_10=bw(df_2009_10)
bw_2010_11=bw(df_2010_11)
bw_2011_12=bw(df_2011_12)
bw_2012_13=bw(df_2012_13)
bw_2013_14=bw(df_2013_14)
bw_2014_15=bw(df_2014_15)
bw_2015_16=bw(df_2015_16)
bw_2016_17=bw(df_2016_17)
bw_2017_18=bw(df_2017_18)
bw_2018_19=bw(df_2018_19)
bw_2019_20=bw(df_2019_20)

## Shots on Target

In [287]:
def shots_on_target(df):    
    home_points=np.zeros(df.shape[0])
    away_points=np.zeros(df.shape[0])
    for i in df.index:
        home_team=df.loc[i,"HomeTeam"]
        away_team=df.loc[i,"AwayTeam"]
        temp1=df.groupby(["AwayTeam"]).get_group((f"{home_team}"))["AST"]
        temp2=df.groupby(["HomeTeam"]).get_group((f"{home_team}"))["HST"]
        temp3=pd.concat([temp1,temp2]).sort_index()
        temp4=temp3.loc[temp3.index[0]:i]
        temp5=temp4.loc[temp4.index[0]:temp4.index[len(temp4)-2]]
        if len(temp5)==1:
            home_points[i]=0
        else:
            home_points[i]=temp5.sum()
        temp5=df.groupby(["AwayTeam"]).get_group((f"{away_team}"))["AST"]
        temp6=df.groupby(["HomeTeam"]).get_group((f"{away_team}"))["HST"]
        temp7=pd.concat([temp5,temp6]).sort_index()
        temp8=temp7.loc[temp7.index[0]:i]
        temp9=temp8.loc[temp8.index[0]:temp8.index[len(temp8)-2]]
        if len(temp9)==1:
            away_points[i]=0
        else:
            away_points[i]=temp9.sum()
    return pd.DataFrame({"home shots on target":home_points,"away shots on target":away_points})

In [288]:
shots_on_target_2009_10=shots_on_target(df_2009_10)
shots_on_target_2010_11=shots_on_target(df_2010_11)
shots_on_target_2011_12=shots_on_target(df_2011_12)
shots_on_target_2012_13=shots_on_target(df_2012_13)
shots_on_target_2013_14=shots_on_target(df_2013_14)
shots_on_target_2014_15=shots_on_target(df_2014_15)
shots_on_target_2015_16=shots_on_target(df_2015_16)
shots_on_target_2016_17=shots_on_target(df_2016_17)
shots_on_target_2017_18=shots_on_target(df_2017_18)
shots_on_target_2018_19=shots_on_target(df_2018_19)
shots_on_target_2019_20=shots_on_target(df_2019_20)

## Results

In [290]:
def result(df):
    result=df[["FTR"]].replace({"H":1,"A":-1,"D":0})
    return result

In [291]:
result_2009_10=result(df_2009_10)
result_2010_11=result(df_2010_11)
result_2011_12=result(df_2011_12)
result_2012_13=result(df_2012_13)
result_2013_14=result(df_2013_14)
result_2014_15=result(df_2014_15)
result_2015_16=result(df_2015_16)
result_2016_17=result(df_2016_17)
result_2017_18=result(df_2017_18)
result_2018_19=result(df_2018_19)
result_2019_20=result(df_2019_20)

## Final Dataframe

In [292]:
df_final_2009_10=pd.concat([bet365_2009_10,home_away_goals_2009_10,form_2009_10,points_2009_10,goals_conceded_2009_10,red_card_last_game_2009_10,bw_2009_10,result_2009_10, shots_on_target_2009_10],axis=1)
df_final_2010_11=pd.concat([bet365_2010_11,home_away_goals_2010_11,form_2010_11,points_2010_11,goals_conceded_2010_11,red_card_last_game_2010_11,bw_2010_11,result_2010_11, shots_on_target_2010_11],axis=1)
df_final_2011_12=pd.concat([bet365_2011_12,home_away_goals_2011_12,form_2011_12,points_2011_12,goals_conceded_2011_12,red_card_last_game_2011_12,bw_2011_12,result_2011_12, shots_on_target_2011_12],axis=1)
df_final_2012_13=pd.concat([bet365_2012_13,home_away_goals_2012_13,form_2012_13,points_2012_13,goals_conceded_2012_13,red_card_last_game_2012_13,bw_2012_13,result_2012_13, shots_on_target_2012_13],axis=1)
df_final_2013_14=pd.concat([bet365_2013_14,home_away_goals_2013_14,form_2013_14,points_2013_14,goals_conceded_2013_14,red_card_last_game_2013_14,bw_2013_14,result_2013_14, shots_on_target_2013_14],axis=1)
df_final_2014_15=pd.concat([bet365_2014_15,home_away_goals_2014_15,form_2014_15,points_2014_15,goals_conceded_2014_15,red_card_last_game_2014_15,bw_2014_15,result_2014_15, shots_on_target_2014_15],axis=1)
df_final_2015_16=pd.concat([bet365_2015_16,home_away_goals_2015_16,form_2015_16,points_2015_16,goals_conceded_2015_16,red_card_last_game_2015_16,bw_2015_16,result_2015_16, shots_on_target_2015_16],axis=1)
df_final_2016_17=pd.concat([bet365_2016_17,home_away_goals_2016_17,form_2016_17,points_2016_17,goals_conceded_2016_17,red_card_last_game_2016_17,bw_2016_17,result_2016_17, shots_on_target_2016_17],axis=1)
df_final_2017_18=pd.concat([bet365_2017_18,home_away_goals_2017_18,form_2017_18,points_2017_18,goals_conceded_2017_18,red_card_last_game_2017_18,bw_2017_18,result_2017_18, shots_on_target_2017_18],axis=1)
df_final_2018_19=pd.concat([bet365_2018_19,home_away_goals_2018_19,form_2018_19,points_2018_19,goals_conceded_2018_19,red_card_last_game_2018_19,bw_2018_19,result_2018_19, shots_on_target_2018_19],axis=1)
df_final_2019_20=pd.concat([bet365_2019_20,home_away_goals_2019_20,form_2019_20,points_2019_20,goals_conceded_2019_20,red_card_last_game_2019_20,bw_2019_20,result_2019_20, shots_on_target_2019_20],axis=1)


In [293]:
x_training=pd.concat([df_final_2009_10,df_final_2010_11,df_final_2011_12,df_final_2012_13,df_final_2013_14,df_final_2014_15,df_final_2015_16])
x_test = pd.concat([df_final_2016_17, df_final_2017_18, df_final_2018_19,df_final_2019_20])



# x_training['home advantage'] = np.full(x_training.shape[0], 5)
# x_training['away disadvantage'] = np.full(x_training.shape[0], 1)
# x_test['home advantage'] = np.full(x_test.shape[0], 5)
# x_test['away disadvantage'] = np.full(x_test.shape[0], 1)

y_training = x_training['FTR']
y_test = x_test['FTR']

x_training.drop('FTR', inplace = True, axis =1 )
x_test.drop('FTR', inplace = True, axis =1 )

x_training.reset_index(inplace=True,drop=True)
x_test.reset_index(inplace=True, drop = True)
y_training.reset_index(inplace = True, drop = True)
y_test.reset_index(inplace = True, drop = True)


y_training.drop([x_training[x_training.isna().any(axis=1)].index[0]], inplace = True)
x_training.dropna(axis = 0, how = 'any', inplace = True)



## Model Predictions



### kNN Classifier

In [ ]:

knn_model = KNeighborsClassifier(n_neighbors = 50, metric = 'euclidean')
knn_model.fit(x_training, y_training)

y_pred_knn = knn_model.predict(x_test)
f1_score = sklearn.metrics.f1_score(y_test, y_pred_knn, average = 'macro')
classification_accuracy = sklearn.metrics.accuracy_score(y_test, y_pred_knn)
confusion_matrix = sklearn.metrics.confusion_matrix(y_test, y_pred_knn)

print("KNN \n")
print("F1 Score:", f1_score)
print("Classification Accuracy:", classification_accuracy)
print("Confusion Matrix:")
ax = sns.heatmap(confusion_matrix, annot=True)





### Logistic Regression

In [ ]:

logistic_reg = LogisticRegression(multi_class = 'ovr', max_iter = 1000)
logistic_reg.fit(x_training, y_training)
y_pred_log = logistic_reg.predict(x_test)

confusion_matrix = metrics.confusion_matrix(y_test, y_pred_log)
f1_score = metrics.f1_score(y_test, y_pred_log, average = 'macro')
classification_accuracy = metrics.accuracy_score(y_test, y_pred_log)

print("Logistic Regression \n")
print("F1 score: ", f1_score)
print("Classification Accuracy: ", classification_accuracy)
print("Confusion Matrix:")
ax = sns.heatmap(confusion_matrix, annot=True)

### Naive Bayes

In [ ]:

NB_classifier = MultinomialNB()
NB_classifier.fit(x_training, y_training)
y_pred_nb = NB_classifier.predict(x_test)

confusion_matrix = metrics.confusion_matrix(y_test, y_pred_nb)
f1_score = metrics.f1_score(y_test, y_pred_nb, average = 'macro')
classification_accuracy = metrics.accuracy_score(y_test, y_pred_nb)

print("Naive Bayes \n")
print("F1 score: ", f1_score)
print("Classification Accuracy: ", classification_accuracy)
print("Confusion Matrix:")
ax = sns.heatmap(confusion_matrix, annot=True)

### SVM

In [ ]:
svm_clf = svm.SVC(kernel = 'linear')
svm_clf.fit(x_training, y_training)
y_pred_svm = svm_clf.predict(x_test)


confusion_matrix = metrics.confusion_matrix(y_test, y_pred_svm)
f1_score = metrics.f1_score(y_test, y_pred_svm, average = 'macro')
classification_accuracy = metrics.accuracy_score(y_test, y_pred_svm)

print("SVM \n")
print("F1 score: ", f1_score)
print("Classification Accuracy: ", classification_accuracy)
print("Confusion Matrix:")
ax = sns.heatmap(confusion_matrix, annot=True)


### Neural Network

In [ ]:

mlp_clf = neural_network.MLPClassifier()
mlp_clf.fit(x_training, y_training)
y_pred_mlp = mlp_clf.predict(x_test)

confusion_matrix = metrics.confusion_matrix(y_test, y_pred_mlp)
f1_score = metrics.f1_score(y_test, y_pred_mlp, average = 'macro')
classification_accuracy = metrics.accuracy_score(y_test, y_pred_mlp)

print("Neural Network \n")
print("F1 score: ", f1_score)
print("Classification Accuracy: ", classification_accuracy)
print("Confusion Matrix:")
ax = sns.heatmap(confusion_matrix, annot=True)
